In [10]:
import os, torch
import numpy as np
from src.coco_eval import load_coco_dataset_in_fiftyone, prepare_dataset_folder
from tqdm.notebook import trange, tqdm
from ssd300_utils import detections_fiftyone_format

from src.utils import selective_l1_loss_indexed, selective_l1_loss
from src.utils import l1_loss


np.set_printoptions(precision=3, linewidth=1000)
torch.set_printoptions(precision=3, linewidth=1000)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

N = 50

### Faster RCNN

In [11]:
%%script echo skipping
from faster_rcnn_utils import *
from torchvision.models.detection import fasterrcnn_resnet50_fpn as FasterRCNN

frcnn = FasterRCNN(pretrained=True).to(device)                                                    # for normal use
# frcnn = FasterRCNN(pretrained=True, box_score_thresh=0, box_detections_per_img=None).to(device)     # for raw outputs
frcnn.training = False
frcnn.eval();


def fgsm(image: torch.tensor, model: torch.nn.Module, steps: int = 5, epsilon: float = 0.1, 
        threshold: float = 0.5, loss_fn: str = 'selective', device: str = 'cuda'):
    # prep targets
    mask = torch.zeros_like(image, requires_grad=True, device=device)
    image.requires_grad = True
    criterion = selective_l1_loss_indexed if loss_fn == 'selective' else l1_loss

    for _ in range(steps):
        out = model(mask + image)[0]

        loss = criterion(out['scores'], threshold=threshold)
        model.zero_grad()
        loss.backward()

        gradient = mask.grad.data.detach()
        mask.data = mask.data - gradient.sign() * epsilon
        mask.grad.data.zero_()
        image.grad.data.zero_()

    # fix the changes
    cloacked_image = (image + mask).detach()

    return cloacked_image

skipping


In [12]:
from models.ssd300.model import SSD, ResNet
from models.ssd300.utils import generate_dboxes, Encoder
from ssd300_utils import *

ssd = SSD(backbone=ResNet(), num_classes=81).to(device)
dboxes = generate_dboxes(model='ssd')
encoder = Encoder(dboxes)

checkpoint = torch.load('data/models/SSD300.pth')
ssd.load_state_dict(checkpoint["model_state_dict"])
ssd.eval();


def fgsm(image: torch.tensor, model: torch.nn.Module, steps: int = 5, epsilon: float = 0.1, 
        threshold: float = 0.5, loss_fn: str = 'selective', device: str = 'cuda'):
    # prep targets
    mask = torch.zeros_like(image, requires_grad=True, device=device)
    image.requires_grad = True
    criterion = selective_l1_loss if loss_fn == 'selective' else l1_loss

    for _ in range(steps):
        _, probs = model((mask + image).contiguous())
        probs = torch.nn.Softmax(dim=1)(probs)[:, 1:, :]
        loss = criterion(probs, threshold=threshold)
        model.zero_grad()
        loss.backward()
        # print((probs > threshold).to(torch.float).sum().sum().sum())

        gradient = mask.grad.data.detach()
        mask.data = mask.data - gradient.sign() * epsilon
        mask.grad.data.zero_()
        image.grad.data.zero_()

    # fix the changes
    cloacked_image = (image + mask).detach()

    return cloacked_image

In [13]:
from src.coco_eval import load_coco_dataset_in_fiftyone, prepare_voc_dataset_folder, load_pascal_dataset_in_fiftyone

# only take a subset of N samples to run the experiments
# dataset = load_coco_dataset_in_fiftyone('data/coco', use_cached=False, max_samples=N)
dataset = load_pascal_dataset_in_fiftyone('data/pascal', use_cached=False, max_samples=N)
img_fnames = [sample['filepath'].split('\\')[-1] for sample in dataset]
del dataset

 100% |███████████████████| 50/50 [121.9ms elapsed, 0s remaining, 414.1 samples/s]    


## Explorative Experiments
### Perturbation Granularity
Study how different step sizes (X) affect the accuracy (Y)  
Compare 2 models / epsilon: pgd_0.10, pgd_0.02 selective_pgd_0.10, selective_pgd_0.02  
Use steps = [1, 5, 10, 20, 50]  

key format = loss_fn, steps, epsilon
granularity_results = {('l1', 1, 0.1): 0.14112058272131758, ('l1', 3, 0.1): 0.08203656917386384, ('l1', 5, 0.1): 0.07281498517659804, ('l1', 10, 0.1): 0.04894273711433147, ('l1', 1, 0.02): 0.2385446840030747, ('l1', 3, 0.02): 0.19749213852173855, ('l1', 5, 0.02): 0.18504676456426883, ('l1', 10, 0.02): 0.2060983347886643, ('selective', 1, 0.1): 0.08907545898492718, ('selective', 3, 0.1): 0.026812255356727845, ('selective', 5, 0.1): 0.029211885395577973, ('selective', 10, 0.1): 0.036954373884075346, ('selective', 1, 0.02): 0.14122894239928338, ('selective', 3, 0.02): 0.11422197961754546, ('selective', 5, 0.02): 0.12945134771476655, ('selective', 10, 0.02): 0.20396236304564416}

In [16]:
params = [ (loss_fn, steps, epsilon)
    for loss_fn in ['l1', 'selective'] 
    for epsilon in [1/10, 1/50]
    for steps in [1, 3, 5, 10]
]
threshold = 0.01

for loss_fn, steps, epsilon in params:
    dst_folder = f'{loss_fn}_{steps}_{epsilon:.3f}'

    # # coco
    # prepare_dataset_folder('data/coco', f'C:/Users/tcilloni/Desktop/coco_exploration/granularity_{dst_folder}')
    # src_fnames = [f'data/coco/images/val2017/{fname}' for fname in img_fnames]
    # dst_fnames = [f'C:/Users/tcilloni/Desktop/coco_exploration/granularity_{dst_folder}/images/val2017/{fname}' for fname in img_fnames]300_{dst_folder}')

    # pascal
    prepare_voc_dataset_folder('data/pascal', f'C:/Users/tcilloni/Desktop/pascal_exploration/granularity_{dst_folder}')
    src_fnames = [f'data/pascal/data/{fname}' for fname in img_fnames]
    dst_fnames = [f'C:/Users/tcilloni/Desktop/pascal_exploration/granularity_{dst_folder}/data/{fname}' for fname in img_fnames]

    # # frcnn
    # for src, dst in tqdm(list(zip(src_fnames, dst_fnames)), desc=dst_folder):
    #     image = image_to_tensor(read_image(src))   
    #     cloacked_image = fgsm(image, frcnn, steps, epsilon / steps, threshold, loss_fn)
    #     save_image(dst, image_to_numpy(cloacked_image))
    
    # ssd
    for src, dst in tqdm(list(zip(src_fnames, dst_fnames)), desc=dst_folder):
        image, dims = read_image(src, shape=(300,300))
        image = image_to_tensor(image)
        cloacked_image = fgsm(image, ssd, steps, 4.4* epsilon / steps, threshold, loss_fn) 
        save_image(dst, image_to_numpy(cloacked_image), dims)

l1_1_0.100:   0%|          | 0/50 [00:00<?, ?it/s]

l1_3_0.100:   0%|          | 0/50 [00:00<?, ?it/s]

l1_5_0.100:   0%|          | 0/50 [00:00<?, ?it/s]

l1_10_0.100:   0%|          | 0/50 [00:00<?, ?it/s]

l1_1_0.020:   0%|          | 0/50 [00:00<?, ?it/s]

...

selective_10_0.100:   0%|          | 0/50 [00:00<?, ?it/s]

selective_1_0.020:   0%|          | 0/50 [00:00<?, ?it/s]

selective_3_0.020:   0%|          | 0/50 [00:00<?, ?it/s]

selective_5_0.020:   0%|          | 0/50 [00:00<?, ?it/s]

selective_10_0.020:   0%|          | 0/50 [00:00<?, ?it/s]

In [17]:
results = {}

for loss_fn, steps, epsilon in params: 
    dst_folder = f'{loss_fn}_{steps}_{epsilon:.3f}'
    # coco_folder = f'C:/Users/tcilloni/Desktop/coco_exploration/granularity_{dst_folder}'
    # dataset = load_coco_dataset_in_fiftyone(coco_folder, use_cached=False, max_samples=N)
    pascal_folder = f'C:/Users/tcilloni/Desktop/pascal_exploration/granularity_{dst_folder}'
    dataset = load_pascal_dataset_in_fiftyone(pascal_folder, use_cached=False, max_samples=N)
    

    for sample in tqdm(dataset, desc=dst_folder):
        # detections = detections_fiftyone_format(sample.filepath, frcnn)
        detections = detections_fiftyone_format(sample.filepath, ssd, encoder, dataset='pascal')
        sample['predictions'] = detections
        sample.save()

    # result = dataset.evaluate_detections('predictions', gt_field='detections', eval_key='eval', compute_mAP=True)       # coco
    result = dataset.evaluate_detections('predictions', gt_field='ground_truth', eval_key='eval', method='open-images') # pascal
    results[(loss_fn, steps, epsilon)] = result.mAP()
    print(f'{dst_folder}\t mAP: {result.mAP():.3f}')

print(results)

 100% |███████████████████| 50/50 [156.8ms elapsed, 0s remaining, 321.2 samples/s]    


l1_1_0.100:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating detections...
 100% |███████████████████| 50/50 [718.8ms elapsed, 0s remaining, 69.6 samples/s]      
l1_1_0.100	 mAP: 0.480
 100% |███████████████████| 50/50 [149.2ms elapsed, 0s remaining, 338.0 samples/s]    


l1_3_0.100:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating detections...
 100% |███████████████████| 50/50 [347.2ms elapsed, 0s remaining, 144.0 samples/s]      
l1_3_0.100	 mAP: 0.493
 100% |███████████████████| 50/50 [150.3ms elapsed, 0s remaining, 334.9 samples/s]    


...

Evaluating detections...
 100% |███████████████████| 50/50 [317.8ms elapsed, 0s remaining, 157.3 samples/s]     
selective_3_0.020	 mAP: 0.504
 100% |███████████████████| 50/50 [151.7ms elapsed, 0s remaining, 332.3 samples/s]    


selective_5_0.020:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating detections...
 100% |███████████████████| 50/50 [307.8ms elapsed, 0s remaining, 162.4 samples/s]      
selective_5_0.020	 mAP: 0.449
 100% |███████████████████| 50/50 [154.5ms elapsed, 0s remaining, 325.8 samples/s]    


selective_10_0.020:   0%|          | 0/50 [00:00<?, ?it/s]

Evaluating detections...
 100% |███████████████████| 50/50 [264.1ms elapsed, 0s remaining, 189.3 samples/s]      
selective_10_0.020	 mAP: 0.452
{('l1', 1, 0.1): 0.48000469755622355, ('l1', 3, 0.1): 0.4933416537406225, ('l1', 5, 0.1): 0.440384693122283, ('l1', 10, 0.1): 0.44943103624991104, ('l1', 1, 0.02): 0.6859845093934132, ('l1', 3, 0.02): 0.6605602125392417, ('l1', 5, 0.02): 0.617218675207466, ('l1', 10, 0.02): 0.619299189446706, ('selective', 1, 0.1): 0.3457165416471404, ('selective', 3, 0.1): 0.23011424090425459, ('selective', 5, 0.1): 0.16158887352277804, ('selective', 10, 0.1): 0.1660643857706281, ('selective', 1, 0.02): 0.45058114733366317, ('selective', 3, 0.02): 0.5042841052568828, ('selective', 5, 0.02): 0.4487580422451335, ('selective', 10, 0.02): 0.4518706824704377}


### Epsilon-Spheres
Study how different step epsilons (X) affect the accuracy (Y)  
Compare 2 models / steps: fgsm, pgd_5, selective_fgsm, selective_pgd_5  
Use epsilon = ```[0.0125, ..., 0.1]```  
Fix threshold = 0.5

key = loss_fn, steps, epsilon
epsilon_data = {('l1', 1, 0.0125): 0.2610616836326749, ('l1', 5, 0.0125): 0.30288166835423025, ('l1', 1, 0.025): 0.2333429904614117, ('l1', 5, 0.025): 0.19620411712033345, ('l1', 1, 0.037500000000000006): 0.21280897334890758, ('l1', 5, 0.037500000000000006): 0.14958786777269148, ('l1', 1, 0.05): 0.19976296463470325, ('l1', 5, 0.05): 0.11604116719582987, ('l1', 1, 0.0625): 0.18356238690367335, ('l1', 5, 0.0625): 0.10281715930727792, ('l1', 1, 0.075): 0.1680623278005409, ('l1', 5, 0.075): 0.0879306138014952, ('l1', 1, 0.08750000000000001): 0.1542645631867448, ('l1', 5, 0.08750000000000001): 0.07705942039886213, ('l1', 1, 0.1): 0.14051193801719752, ('l1', 5, 0.1): 0.07326406004153871, ('selective', 1, 0.0125): 0.1611656643746777, ('selective', 5, 0.0125): 0.2684504775576304, ('selective', 1, 0.025): 0.13496475069067954, ('selective', 5, 0.025): 0.1411491130747048, ('selective', 1, 0.037500000000000006): 0.1252182831529849, ('selective', 5, 0.037500000000000006): 0.08459518107910977, ('selective', 1, 0.05): 0.1139908846362351, ('selective', 5, 0.05): 0.05219425845326645, ('selective', 1, 0.0625): 0.10668323815580096, ('selective', 5, 0.0625): 0.03936162397196825, ('selective', 1, 0.075): 0.09994644467346478, ('selective', 5, 0.075): 0.03050446025801265, ('selective', 1, 0.08750000000000001): 0.0955911895395066, ('selective', 5, 0.08750000000000001): 0.02748972543325359, ('selective', 1, 0.1): 0.08921274261951055, ('selective', 5, 0.1): 0.02516477871738585}

In [7]:
params = [ (loss_fn, steps, epsilon)
    for loss_fn in ['l1', 'selective']  
    for epsilon in np.arange(0.0125, 0.101, 0.0125)
    for steps in [1, 5]  
]
threshold = 0.5

for loss_fn, steps, epsilon in params:
    dst_folder = f'{loss_fn}_{steps}_{epsilon:.4f}'
    prepare_dataset_folder('data/coco', f'C:/Users/tcilloni/Desktop/coco_exploration/epsilon_{dst_folder}')

    src_fnames = [f'data/coco/images/val2017/{fname}' for fname in img_fnames]
    dst_fnames = [f'C:/Users/tcilloni/Desktop/coco_exploration/epsilon_{dst_folder}/images/val2017/{fname}' for fname in img_fnames]

    # # frcnn
    # for src, dst in tqdm(list(zip(src_fnames, dst_fnames)), desc=dst_folder):
    #     image = image_to_tensor(read_image(src))
    #     cloacked_image = fgsm(image, frcnn, steps, epsilon / steps, threshold, loss_fn)
    #     save_image(dst, image_to_numpy(cloacked_image))
    
    # ssd
    for src, dst in tqdm(list(zip(src_fnames, dst_fnames)), desc=dst_folder):
        image, dims = read_image(src, shape=(300,300))
        image = image_to_tensor(image)
        cloacked_image = fgsm(image, ssd, steps, 4.4* epsilon / steps, threshold, loss_fn) 
        save_image(dst, image_to_numpy(cloacked_image), dims)

l1_1_0.0125:   0%|          | 0/250 [00:00<?, ?it/s]

l1_5_0.0125:   0%|          | 0/250 [00:00<?, ?it/s]

l1_1_0.0250:   0%|          | 0/250 [00:00<?, ?it/s]

l1_5_0.0250:   0%|          | 0/250 [00:00<?, ?it/s]

l1_1_0.0375:   0%|          | 0/250 [00:00<?, ?it/s]

...

selective_5_0.0750:   0%|          | 0/250 [00:00<?, ?it/s]

selective_1_0.0875:   0%|          | 0/250 [00:00<?, ?it/s]

selective_5_0.0875:   0%|          | 0/250 [00:00<?, ?it/s]

selective_1_0.1000:   0%|          | 0/250 [00:00<?, ?it/s]

selective_5_0.1000:   0%|          | 0/250 [00:00<?, ?it/s]

In [8]:
results = {}

for loss_fn, steps, epsilon in params: 
    dst_folder = f'{loss_fn}_{steps}_{epsilon:.4f}'
    coco_folder = f'C:/Users/tcilloni/Desktop/coco_exploration/epsilon_{dst_folder}'
    dataset = load_coco_dataset_in_fiftyone(coco_folder, use_cached=False, max_samples=N)

    # frcnn
    for sample in tqdm(dataset, desc=dst_folder):
        # detections = detections_fiftyone_format(sample.filepath, frcnn)
        detections = detections_fiftyone_format(sample.filepath, ssd, encoder)
        sample['predictions'] = detections
        sample.save()

    result = dataset.evaluate_detections('predictions', gt_field='detections', eval_key='eval', compute_mAP=True)
    results[(loss_fn, steps, epsilon)] = result.mAP()
    print(f'{dst_folder}\t mAP: {result.mAP():.3f}')

print(results)

 100% |█████████████████| 250/250 [1.1s elapsed, 0s remaining, 223.5 samples/s]         


l1_1_0.0125:   0%|          | 0/250 [00:00<?, ?it/s]

Evaluating detections...
 100% |█████████████████| 250/250 [5.9s elapsed, 0s remaining, 56.6 samples/s]       
Performing IoU sweep...
 100% |█████████████████| 250/250 [10.9s elapsed, 0s remaining, 29.2 samples/s]      
l1_1_0.0125	 mAP: 0.240
 100% |█████████████████| 250/250 [938.3ms elapsed, 0s remaining, 266.4 samples/s]      


l1_5_0.0125:   0%|          | 0/250 [00:00<?, ?it/s]

Evaluating detections...
 100% |█████████████████| 250/250 [5.6s elapsed, 0s remaining, 62.2 samples/s]       
Performing IoU sweep...
 100% |█████████████████| 250/250 [10.1s elapsed, 0s remaining, 33.0 samples/s]      
l1_5_0.0125	 mAP: 0.235
 100% |█████████████████| 250/250 [1.1s elapsed, 0s remaining, 230.9 samples/s]         


...

Evaluating detections...
 100% |█████████████████| 250/250 [8.3s elapsed, 0s remaining, 36.3 samples/s]       
Performing IoU sweep...
 100% |█████████████████| 250/250 [15.3s elapsed, 0s remaining, 15.1 samples/s]      
selective_5_0.0875	 mAP: 0.085
 100% |█████████████████| 250/250 [950.6ms elapsed, 0s remaining, 263.5 samples/s]      


selective_1_0.1000:   0%|          | 0/250 [00:00<?, ?it/s]

Evaluating detections...
 100% |█████████████████| 250/250 [6.8s elapsed, 0s remaining, 46.5 samples/s]       
Performing IoU sweep...
 100% |█████████████████| 250/250 [12.7s elapsed, 0s remaining, 22.5 samples/s]      
selective_1_0.1000	 mAP: 0.094
 100% |█████████████████| 250/250 [1.1s elapsed, 0s remaining, 233.7 samples/s]         


selective_5_0.1000:   0%|          | 0/250 [00:00<?, ?it/s]

Evaluating detections...
 100% |█████████████████| 250/250 [8.2s elapsed, 0s remaining, 35.0 samples/s]       
Performing IoU sweep...
 100% |█████████████████| 250/250 [15.3s elapsed, 0s remaining, 15.2 samples/s]      
selective_5_0.1000	 mAP: 0.078
{('l1', 1, 0.0125): 0.23956523216831016, ('l1', 5, 0.0125): 0.23517860212129502, ('l1', 1, 0.025): 0.2198413949026522, ('l1', 5, 0.025): 0.20346247597425196, ('l1', 1, 0.037500000000000006): 0.20551035858936081, ('l1', 5, 0.037500000000000006): 0.18719932815969606, ('l1', 1, 0.05): 0.19205961252693432, ('l1', 5, 0.05): 0.16887228981789043, ('l1', 1, 0.0625): 0.18104334963403992, ('l1', 5, 0.0625): 0.16290905350931886, ('l1', 1, 0.075): 0.16937718133148033, ('l1', 5, 0.075): 0.14309231683525675, ('l1', 1, 0.08750000000000001): 0.15861490406857617, ('l1', 5, 0.08750000000000001): 0.14135033217473397, ('l1', 1, 0.1): 0.1442887934758087, ('l1', 5, 0.1): 0.1306817261501141, ('selective', 1, 0.0125): 0.14303497213958782, ('selective', 5, 0.0125

### Focusing Threshold
Study how different thresholds (X) affect the accuracy (Y)  
Compare 2 models: fgsm, pgd, selective fgsm, selective pgd  
Fix epsilon = 0.02

In [9]:
params = [ (loss_fn, steps, threshold)
    for loss_fn in ['selective']  
    # for loss_fn in ['l1', 'selective']  
    for threshold in np.arange(0, 0.99, 0.1)
    for steps in [1, 5]  
]
epsilon = 1 / 50

for loss_fn, steps, threshold in params:
    dst_folder = f'{loss_fn}_{steps}_{threshold:.1f}'
    prepare_dataset_folder('data/coco', f'C:/Users/tcilloni/Desktop/coco_exploration/threshold_{dst_folder}')

    src_fnames = [f'data/coco/images/val2017/{fname}' for fname in img_fnames]
    dst_fnames = [f'C:/Users/tcilloni/Desktop/coco_exploration/threshold_{dst_folder}/images/val2017/{fname}' for fname in img_fnames]

    # # frcnn
    # for src, dst in tqdm(list(zip(src_fnames, dst_fnames)), desc=dst_folder):
    #     image = image_to_tensor(read_image(src))
    #     cloacked_image = fgsm(image, frcnn, steps, epsilon / steps, threshold, loss_fn)
    #     save_image(dst, image_to_numpy(cloacked_image))
    
    # ssd
    for src, dst in tqdm(list(zip(src_fnames, dst_fnames)), desc=dst_folder):
        image, dims = read_image(src, shape=(300,300))
        image = image_to_tensor(image)
        cloacked_image = fgsm(image, ssd, steps, 4.4* epsilon / steps, threshold, loss_fn) 
        save_image(dst, image_to_numpy(cloacked_image), dims)

selective_1_0.0:   0%|          | 0/250 [00:00<?, ?it/s]

selective_5_0.0:   0%|          | 0/250 [00:00<?, ?it/s]

selective_1_0.1:   0%|          | 0/250 [00:00<?, ?it/s]

selective_5_0.1:   0%|          | 0/250 [00:00<?, ?it/s]

selective_1_0.2:   0%|          | 0/250 [00:00<?, ?it/s]

...

selective_5_0.7:   0%|          | 0/250 [00:00<?, ?it/s]

selective_1_0.8:   0%|          | 0/250 [00:00<?, ?it/s]

selective_5_0.8:   0%|          | 0/250 [00:00<?, ?it/s]

selective_1_0.9:   0%|          | 0/250 [00:00<?, ?it/s]

selective_5_0.9:   0%|          | 0/250 [00:00<?, ?it/s]

In [10]:
%%script echo skipping
import time

loss_fn = 'selective'
epsilon = 1 / 50
image = image_to_tensor(read_image(src))
iters = 100

for steps in [1, 5]:
    for threshold in [0] + list(np.arange(0.1, 0.99, 0.2)):
        t = time.time()

        for _ in range(iters):
            fgsm(image, frcnn, steps, epsilon / steps, threshold, loss_fn)
        
        et = (time.time() - t) / iters
        print(f'{steps} - {threshold:.1f}: {et:.3f}')

skipping


In [11]:
results = {}

for loss_fn, steps, threshold in params: 
    dst_folder = f'{loss_fn}_{steps}_{threshold:.1f}'
    coco_folder = f'C:/Users/tcilloni/Desktop/coco_exploration/threshold_{dst_folder}'
    dataset = load_coco_dataset_in_fiftyone(coco_folder, use_cached=False, max_samples=N)

    for sample in tqdm(dataset, desc=dst_folder):
        # detections = detections_fiftyone_format(sample.filepath, frcnn)
        detections = detections_fiftyone_format(sample.filepath, ssd, encoder)
        sample['predictions'] = detections
        sample.save()

    result = dataset.evaluate_detections('predictions', gt_field='detections', eval_key='eval', compute_mAP=True)
    results[(loss_fn, steps, threshold)] = result.mAP()
    print(f'{dst_folder}\t mAP: {result.mAP():.3f}')

print(results)

 100% |█████████████████| 250/250 [926.5ms elapsed, 0s remaining, 269.8 samples/s]      


selective_1_0.0:   0%|          | 0/250 [00:00<?, ?it/s]

Evaluating detections...
 100% |█████████████████| 250/250 [5.5s elapsed, 0s remaining, 63.6 samples/s]       
Performing IoU sweep...
 100% |█████████████████| 250/250 [9.9s elapsed, 0s remaining, 35.3 samples/s]       
selective_1_0.0	 mAP: 0.231
 100% |█████████████████| 250/250 [1.1s elapsed, 0s remaining, 231.5 samples/s]         


selective_5_0.0:   0%|          | 0/250 [00:00<?, ?it/s]

Evaluating detections...
 100% |█████████████████| 250/250 [4.4s elapsed, 0s remaining, 70.1 samples/s]       
Performing IoU sweep...
 100% |█████████████████| 250/250 [8.0s elapsed, 0s remaining, 46.1 samples/s]       
selective_5_0.0	 mAP: 0.205
 100% |█████████████████| 250/250 [931.8ms elapsed, 0s remaining, 268.3 samples/s]      


...

Evaluating detections...
 100% |█████████████████| 250/250 [8.7s elapsed, 0s remaining, 34.9 samples/s]       
Performing IoU sweep...
 100% |█████████████████| 250/250 [16.2s elapsed, 0s remaining, 14.6 samples/s]      
selective_5_0.8	 mAP: 0.209
 100% |█████████████████| 250/250 [1.1s elapsed, 0s remaining, 225.0 samples/s]         


selective_1_0.9:   0%|          | 0/250 [00:00<?, ?it/s]

Evaluating detections...
 100% |█████████████████| 250/250 [8.5s elapsed, 0s remaining, 36.2 samples/s]       
Performing IoU sweep...
 100% |█████████████████| 250/250 [15.7s elapsed, 0s remaining, 15.3 samples/s]      
selective_1_0.9	 mAP: 0.192
 100% |█████████████████| 250/250 [930.4ms elapsed, 0s remaining, 268.7 samples/s]      


selective_5_0.9:   0%|          | 0/250 [00:00<?, ?it/s]

Evaluating detections...
 100% |█████████████████| 250/250 [8.7s elapsed, 0s remaining, 35.4 samples/s]       
Performing IoU sweep...
 100% |█████████████████| 250/250 [16.6s elapsed, 0s remaining, 12.9 samples/s]      
selective_5_0.9	 mAP: 0.241
{('selective', 1, 0.0): 0.23066128552115187, ('selective', 5, 0.0): 0.20512701874160783, ('selective', 1, 0.1): 0.10866471535857831, ('selective', 5, 0.1): 0.09238662038793477, ('selective', 1, 0.2): 0.10792264020612533, ('selective', 5, 0.2): 0.12224354507082301, ('selective', 1, 0.30000000000000004): 0.11070843875690584, ('selective', 5, 0.30000000000000004): 0.13359095071056853, ('selective', 1, 0.4): 0.11956006810634272, ('selective', 5, 0.4): 0.15031052778355428, ('selective', 1, 0.5): 0.12302303512247648, ('selective', 5, 0.5): 0.16428598497434813, ('selective', 1, 0.6000000000000001): 0.12723526837802676, ('selective', 5, 0.6000000000000001): 0.17799294454742645, ('selective', 1, 0.7000000000000001): 0.14065546393061532, ('selective',

### Number of steps + runtime + threshold
Study how different methods (rows) and steps (cols) affect the accuracy / speed  
Compare 4 steps: 1, 5, 10, 20  
Compare 2 models: L1, focused L1  
Use fixed epsilon = 1 / 50  

In [12]:
%%script echo skipping
src_fnames = [f'data/coco/images/val2017/{fname}' for fname in img_fnames]

for loss_fn, steps, threshold in params:
    for src in tqdm(src_fnames, desc=dst_folder):
        image = image_to_tensor(read_image(src))
        cloacked_image = fgsm(image, frcnn, steps, epsilon / steps, threshold, loss_fn)

skipping


## Perceptibility

In [13]:
from src.coco_eval import load_coco_dataset_in_fiftyone
from faster_rcnn_utils import *
from PIL import Image
import numpy as np
import math
from tqdm.notebook import tqdm

In [14]:
read_image = lambda fname : np.array(Image.open(fname).convert('RGB')).astype(np.int32)
pixels_in = lambda array : math.prod(array.shape)

orig_folder = 'data/coco/images/val2017'
pert_folder = 'C:/Users/tcilloni/Desktop/coco_cloaked'
img_fnames = [fname for fname in os.listdir(orig_folder)]

differences = {}

for model in ['frcnn', 'ssd300', 'retinanet'][0:1]:
    for loss in ['l1', 'selective']:
        for attack in ['fgsm', 'pgd']:
            for epsilon in ['0.100', '0.020'][1:]:
                subfolder = f'{model}_{loss}_{attack}_{epsilon}/images/val2017'
                differences[(loss, attack)] = []

                for fname in tqdm(img_fnames, desc=f'{model}_{loss}_{attack}_{epsilon}'):
                    orig_fname = f'{orig_folder}/{fname}'
                    pert_fname = f'{pert_folder}/{subfolder}/{fname}'

                    orig_image = read_image(orig_fname)
                    pert_image = read_image(pert_fname)
                    mean_l1 = abs(orig_image - pert_image).mean()                                    # mean L1 distance
                    # mean_l2 = np.linalg.norm(abs(orig_image - pert_image)) / pixels_in(orig_image)   # mean L2 distance                    
                    differences[(loss, attack)].append(mean_l1)

np.save('results/differences.npy', differences)

frcnn_l1_fgsm_0.020:   0%|          | 0/5000 [00:00<?, ?it/s]

frcnn_l1_pgd_0.020:   0%|          | 0/5000 [00:00<?, ?it/s]

frcnn_selective_fgsm_0.020:   0%|          | 0/5000 [00:00<?, ?it/s]

frcnn_selective_pgd_0.020:   0%|          | 0/5000 [00:00<?, ?it/s]